# Imviz Demonstration Notebook


This notebook demonstrates the Imviz API in the Notebook setting. UI equivalents for these actions, as well as additional documentation about Imviz, can be found here: https://jdaviz.readthedocs.io/en/latest/imviz/

Import modules needed for this notebook.

In [ ]:
import warnings

import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.table import Table
from photutils.aperture import CircularAperture, SkyCircularAperture
from regions import PixCoord, CirclePixelRegion, CircleSkyRegion

from jdaviz import Imviz

We create an Imviz instance and grab the default viewer instance as well.

In [ ]:
imviz = Imviz()
viewer = imviz.default_viewer

Now we load observations. If you already have the files on your local machine, you can 
load them by passing their file paths to `load_data` as strings. For this example, 
we will retrieve remote data from [MAST](https://mast.stsci.edu/) via `astroquery`
by passing the observation's URI as a string. By default, the downloaded files are 
saved to the current working directory. If you set `cache=False` in the `load_data` 
call, it will re-download the file if desired.

In this example, we use two of the JWST NIRCam
observations of the Cartwheel Galaxy. These are large files (~500 MB each), so they make take
some time to download, but after the first time running this notebook they will be cached locally.
If you want to look at images from other filters, you can uncomment the lines with those filenames.
If you want an example using smaller files, see the ImvizDitheredExample notebook, which
includes the same workflow as this one, but using two HST ACS files that are much smaller
than the JWST files used here.

After downloading, each file is loaded into Imviz. We ignore some warnings that appear during parsing.

In [ ]:
filenames = [
    'jw02727-o002_t062_nircam_clear-f090w_i2d.fits',
    # 'jw02727-o002_t062_nircam_clear-f150w_i2d.fits',
    # 'jw02727-o002_t062_nircam_clear-f200w_i2d.fits',
    'jw02727-o002_t062_nircam_clear-f277w_i2d.fits',
    # 'jw02727-o002_t062_nircam_clear-f356w_i2d.fits',
    # 'jw02727-o002_t062_nircam_clear-f444w_i2d.fits'
]

with warnings.catch_warnings(), imviz.batch_load():
    warnings.simplefilter('ignore')
    for fn in filenames:
        uri = f"mast:JWST/product/{fn}"
        imviz.load(uri, cache=True)

Then, we visualize the data and start off by looking at some of the basic features:

In [ ]:
imviz.show()

Alternatively, the data and the configuration can be autodetected and loaded simultaneously by calling `jdaviz.open(f'{data_dir}/{fn}')`

The first thing you will probably notice is that the image doesn't take up the entire viewer area. 
If you press the "b" key to blink to the other loaded image, you will see that this image is 
zoomed correctly by default. The odd default zoom on the other is because the images are linked
by pixel when loaded. We can instead link by WCS (world coordinates) so that the images
will be properly aligned:

In [ ]:
imviz.link_data(align_by='wcs')

Panning and zooming is possible by showing the viewer toolbar and clicking on the '+'-shaped icon, then dragging around in the image and using scrolling to zoom in and out. To change the stretch and colormap, show the **Layer** options accessible through the last icon in the viewer toolbar.

We can also change these programmatically, for example the stretch:

In [ ]:
viewer.stretch_options

In [ ]:
viewer.stretch = 'sqrt'

the colormap:

In [ ]:
viewer.colormap_options

In [ ]:
viewer.set_colormap('Viridis')

the limits via the autocut using percentile option:

In [ ]:
viewer.autocut_options

In [ ]:
viewer.cuts = '95%'
viewer.cuts

or the limits directly:

In [ ]:
viewer.cuts = (0, 2)

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to Astropy regions. Click on the viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = imviz.plugins["Subset Tools"].get_regions()

In [ ]:
regions

It is possible to programmatically pass a `regions` shape, a `photutils` aperture shape, or a Numpy mask into Imviz.

In [ ]:
c = SkyCoord('00h37m41.1171s -33d42m58.5745s')

# photutils aperture
my_aper = CircularAperture((1714, 1380), r=20)
my_aper_sky = SkyCircularAperture(c, 2 * u.arcsec)

# regions shape
my_reg = CirclePixelRegion(center=PixCoord(x=1526, y=1063), radius=40)
my_reg_sky = CircleSkyRegion(c, Angle(2, u.arcsec))

my_regions = [my_aper, my_aper_sky, my_reg, my_reg_sky]
imviz.plugins['Subset Tools'].import_region(my_regions, combination_mode='new')

You can also programmatically control the viewer. Note that the following will fail
if you have blinked between the two images and have data `B` active, due to the
smaller size of that image.

In [ ]:
# Center the image on given pixel position.
viewer.center_on((1900, 2500))  # X, Y (0-indexed)

In [ ]:
# Move the image with the given pixel offsets.
viewer.offset_by(500, -100)  # dX, dY

In [ ]:
# Center the image on given sky coordinates.
sky = SkyCoord('00h37m41.1171s -33d42m58.5745s')
viewer.center_on(sky)

In [ ]:
# Move the image with the given sky offsets.
viewer.offset_by(0.5 * u.arcsec, -1.5 * u.arcsec)

You can programmatically zoom in and out.

Zoom level:

* 1 means real-pixel-size.
* 2 means zoomed in by a factor of 2.
* 0.5 means zoomed out by a factor of 2.
* 'fit' means zoomed to fit the whole image width into display.

In [ ]:
# Get the current zoom level.
viewer.zoom_level

In [ ]:
# Set the zoom level directly.
viewer.zoom_level = 1

In [ ]:
# Set the relative zoom based on current zoom level.
viewer.zoom(2)

It is also possible to programmatically add non-interactive markers to the image.

In [ ]:
# Add 20 randomly generated X,Y (0-indexed) w.r.t. reference image
# using default marker properties.
t_xy = Table({'x': np.random.randint(0, 4000, 20),
              'y': np.random.randint(0, 4000, 20)})
viewer.add_markers(t_xy)

You could customize marker color, alpha, size, and fill with values that Glue supports.

In [ ]:
viewer.marker = {'color': 'green', 'alpha': 0.8, 'markersize': 10, 'fill': False}

In [ ]:
# Mark some sky coordinates using updated marker properties.
t_sky = Table({'coord': [SkyCoord('00h37m42.278s -33d42m00.271s'),
                         SkyCoord('00h37m40.111s -33d42m36.871s'),
                         SkyCoord('00h37m43.411s -33d42m55.64s'),
                         SkyCoord('00h37m41.611s -33d42m56.15s')]})
viewer.add_markers(t_sky, use_skycoord=True, marker_name='my_sky')
t_sky

When you do not need the markers anymore, you could remove them by name.

In [ ]:
viewer.remove_markers(marker_name='my_sky')

You can also remove all the markers.

In [ ]:
viewer.reset_markers()

You can save the active display as PNG file.

In [ ]:
viewer.save('myimage.png')

If you want a second viewer, run the following cell *or* click on the "picture with +" icon on top of the image viewer.

If you click the icon, the viewer name will be auto-generated. Instead of running the next cell, you can run this instead:

    viewer_2_name = 'imviz-1'
    viewer_2 = imviz.app.get_viewer_by_id(viewer_2_name)

In [ ]:
viewer_2_name = 'my-viewer-1'

# Do not pass in viewer_name if you want it auto-generated.
# If auto-generated, viewer name will be 'imviz-1'.
viewer_2 = imviz.create_image_viewer(viewer_name=viewer_2_name)

You can also programmatically control the new viewer, as follows.

In [ ]:
# Show the first image in the second viewer.
imviz.app.add_data_to_viewer(viewer_2_name, 'jw02727-o002_t062_nircam_clear-f277w_i2d[SCI,1]')

In [ ]:
viewer_2.zoom(4)

In [ ]:
viewer_2.cuts = '95%'

In [ ]:
viewer_2.center_on((995, 1000))

To destroy this new viewer, run the following cell *or* click the "x" that is right next to the viewer ID.

In [ ]:
imviz.destroy_viewer(viewer_2_name)